[View in Colaboratory](https://colab.research.google.com/github/StefOe/all-conv-pytorch/blob/master/cifar10.ipynb)

In [5]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = False

In [6]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [7]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [8]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

In [9]:
try:
    from allconv import AllConvNet
except ModuleNotFoundError: 
    !wget https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
    from allconv import AllConvNet

--2018-08-15 01:10:03--  https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
Resolviendo github.com (github.com)... 192.30.253.113, 192.30.253.112
Conectando con github.com (github.com)[192.30.253.113]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Ubicación: https://raw.githubusercontent.com/StefOe/all-conv-pytorch/8545bb3ff7ac4410e52d8db930f0485074073ea9/allconv.py [siguiente]
--2018-08-15 01:10:05--  https://raw.githubusercontent.com/StefOe/all-conv-pytorch/8545bb3ff7ac4410e52d8db930f0485074073ea9/allconv.py
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[151.101.216.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 1457 (1,4K) [text/plain]
Grabando a: “allconv.py”

allconv.py          100%[===================>]   1,42K  --.-KB/s    in 0s      

2018-08-15 01:10:05 (677 MB/s) - “allconv.py” guardado [1457/

In [ ]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.train_data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.train_data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

In [14]:
model = AllConvNet(3)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [15]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        if gsync_save:
            gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [16]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

/home/facundo/.python/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.306105
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.301137
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.304296
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.303145
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.302789
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.301798
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.301848
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.305042
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.301738
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.303952
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.300661
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.303594
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.296668


Process Process-136:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/facundo/.python/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/facundo/.python/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/facundo/.python/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/facundo/.python/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/facundo/.python/lib/python3.6/site-packages/torchvision/datasets/cifar.py", line 121, in __getitem__
    img = self.transform(img)
  File "/home/facundo/.python/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/facundo/.python/lib/python3.6/site-packages/torchvision/transforms/t

In [0]:
# if in Google Colab, download your model with this
from google.colab import files
files.download("best.pt")